In [0]:
!pip install simpy

In [0]:
import simpy as sm
import random as rm
import numpy as np
import math

In [0]:
RANDOM_SEED = 978
number_of_calls = 5000
number_of_calls_spotted = 0
interarrival_mean = 14.3
dummy_mean = 7.2
dummy_std = 2.7
def lognormal2normal(u, v):
    return math.log(u ** 2 / math.sqrt(v + u ** 2)), math.sqrt(math.log(1 + v / u ** 2))
dummy_normal_mean, dummy_normal_std = lognormal2normal(dummy_mean, dummy_std ** 2)
reneging_mean = 60
expert_service_mean = 10.2
expert_break_dur = 3
expert_break_mean = 60

In [0]:
def get_break_interarrival_time():
    return rm.expovariate(1 / expert_break_mean)

class Call:
    next_break_time = get_break_interarrival_time()
    last_call_finish_time = 0
    exited_call_number = 0
    
    def __init__(self, name, id, env, dummy, expert):
        self.env = env
        self.name = name
        self.id = id
        self.arrival_t = self.env.now
        self.arrival_to_expert = None
        self.patience = rm.expovariate(1 / reneging_mean)
        self.action = env.process(self.call())
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
        global number_of_calls_spotted
        number_of_calls_spotted += 1
 
        with dummy.request() as req:
            yield req
            print('%s is assigned to dummy at %g' % (self.name, self.env.now))
            dummy_queue_wait_times[self.id] = self.env.now - self.arrival_t
            yield self.env.process(self.dummy_service())
            print('%s is done with dummy at %g' % (self.name, self.env.now))
            self.arrival_to_expert = self.env.now
            
        with expert.request() as req:
            yield req
            expert_queue_wait_times[self.id] = self.env.now - self.arrival_to_expert
            temp_ex_q_w = expert_queue_wait_times[self.id]
            expert_patiences[self.id] = self.patience
            while self.arrival_to_expert > Call.next_break_time:
                yield env.timeout(max(3 - (self.env.now - Call.last_call_finish_time), 0))
                Call.last_call_finish_time = Call.next_break_time
                Call.next_break_time += get_break_interarrival_time()
                print('expert took a break at %g' % Call.last_call_finish_time)
            print('%s is assigned to expert at %g' % (self.name, self.env.now))
            if temp_ex_q_w <= self.patience:
                yield self.env.process(self.expert_service())
                print('%s is done with expert at %g' % (self.name, self.env.now))
            else:
                print('%s has run out of patience at %g' % (self.name, self.env.now - temp_ex_q_w + self.patience))
            Call.exited_call_number += 1
            if Call.exited_call_number >= number_of_calls:
                raise Exception
            Call.last_call_finish_time = self.env.now
            
    def dummy_service(self):
        duration = rm.lognormvariate(dummy_normal_mean, dummy_normal_std)
        yield self.env.timeout(duration)
        dummy_service_times[self.id] = duration
        
    def expert_service(self):
        duration = rm.expovariate(1 / expert_service_mean)
        yield self.env.timeout(duration)
        expert_service_times[self.id] = duration

In [0]:
def call_generator(env, dummy, expert, N):
    for i in range(N):
        yield env.timeout(rm.expovariate(1 / interarrival_mean))
        call = Call('Call %s' %(i+1), i,  env, dummy, expert)

In [0]:
rm.seed(RANDOM_SEED)

dummy_service_times = {}
dummy_queue_wait_times = {}
expert_service_times = {}
expert_queue_wait_times = {}
expert_patiences = {}

try:
    env = sm.Environment()
    dummy = sm.Resource(env, capacity = 1)
    expert = sm.Resource(env, capacity = 1)
    env.process(call_generator(env, dummy, expert, number_of_calls+10000))
    env.run()
except:
    pass


Call 1 initiated a call at 63.1908
Call 1 is assigned to dummy at 63.1908
Call 2 initiated a call at 64.5864
Call 1 is done with dummy at 73.2782
Call 2 is assigned to dummy at 73.2782
expert took a break at 33.799
expert took a break at 72.1318
Call 1 is assigned to expert at 73.2782
Call 2 is done with dummy at 84.6306
Call 3 initiated a call at 88.1814
Call 3 is assigned to dummy at 88.1814
Call 3 is done with dummy at 93.5318
Call 1 is done with expert at 93.5411
expert took a break at 76.5501
Call 2 is assigned to expert at 96.5411
Call 2 is done with expert at 109.85
Call 3 is assigned to expert at 109.85
Call 3 has run out of patience at 105.998
Call 4 initiated a call at 121.843
Call 4 is assigned to dummy at 121.843
Call 4 is done with dummy at 128.003
Call 4 is assigned to expert at 128.003
Call 4 is done with expert at 135.526
Call 5 initiated a call at 141.876
Call 5 is assigned to dummy at 141.876
Call 5 is done with dummy at 148.038
Call 5 is assigned to expert at 148.038

In [0]:
np_dummy_service_times = np.array([dummy_service_times[i] if i in dummy_service_times else 0 for i in range(number_of_calls_spotted)])
np_dummy_queue_wait_times = np.array([dummy_queue_wait_times[i] if i in dummy_queue_wait_times else 0 for i in range(number_of_calls_spotted)])
np_expert_service_times = np.array([expert_service_times[i] if i in expert_service_times else 0 for i in range(number_of_calls_spotted)])
np_expert_queue_wait_times = np.array([expert_queue_wait_times[i] if i in expert_queue_wait_times else 0 for i in range(number_of_calls_spotted)])
np_expert_patiences = np.array([expert_patiences[i] if i in expert_patiences else 0 for i in range(number_of_calls_spotted)])
not_reneged = np_expert_queue_wait_times <= np_expert_patiences
expert_real_queue_wait_times = np.minimum(np_expert_queue_wait_times, np_expert_patiences)

print('Front desk operator utilization is %f' % (np.sum(np_dummy_service_times) / env.now))
print('Expert utilization is %f' % (np.sum(np_expert_service_times) / env.now))
print('Average total waiting time is %f' % (np.sum((np_dummy_queue_wait_times + expert_real_queue_wait_times)[: number_of_calls]) / number_of_calls))
print('Maximum total waiting time to total system time ratio is %f' % (np.amax((np_dummy_queue_wait_times + expert_real_queue_wait_times)[: number_of_calls] / (np_dummy_queue_wait_times + np_dummy_service_times + expert_real_queue_wait_times + np_expert_service_times)[: number_of_calls])))
print('Average number of people waiting to be served by the expert operator is %f' % (np.sum(expert_real_queue_wait_times) / env.now))

#print(number_of_calls_spotted)
#print(len(dummy_service_times))
#print(len(expert_service_times))
#print(len(dummy_queue_wait_times))
#print(len(expert_queue_wait_times))

Front desk operator utilization is 0.503463
Expert utilization is 0.611428
Average total waiting time is 12.427572
Maximum total waiting time to total system time ratio is 0.959747
Average number of people waiting to be served by the expert operator is 0.575835
